In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
from keras.preprocessing import image                  
from tqdm import tqdm
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as preprocess_input_vgg19
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers import Input, Dense
from keras.layers.core import Dropout, Activation
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from sklearn.metrics import accuracy_score


def load_dataset(path):
    data = load_files(path)
    ship_files = np.array(data['filenames'])
    ship_targets = np_utils.to_categorical(np.array(data['target']), 12)
    return ship_files, ship_targets

train_files, train_targets = load_dataset('data/images/train')
valid_files, valid_targets = load_dataset('data/images/valid')
test_files, test_targets = load_dataset('data/images/test')

ship_names = [item[20:-1] for item in sorted(glob("data/images/train/*/"))]

# Let's check the dataset
print('There are %d total vessel categories.' % len(ship_names))
print('There are %s total vessel images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training vessel images.' % len(train_files))
print('There are %d validation vessel images.' % len(valid_files))
print('There are %d test vessel images.'% len(test_files))

Using TensorFlow backend.


There are 12 total vessel categories.
There are 9825 total vessel images.

There are 7425 training vessel images.
There are 1200 validation vessel images.
There are 1200 test vessel images.


In [2]:
#using machinememo blog to do prediction
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

def extract_VGG19(file_paths):
    tensors = paths_to_tensor(file_paths).astype('float32')
    preprocessed_input = preprocess_input_vgg19(tensors)
    return VGG19(weights='imagenet', include_top=False).predict(preprocessed_input, batch_size=32)

def extract_Resnet50(file_paths):
    tensors = paths_to_tensor(file_paths).astype('float32')
    preprocessed_input = preprocess_input_resnet50(tensors)
    return ResNet50(weights='imagenet', include_top=False).predict(preprocessed_input, batch_size=32)

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

train_vgg19 = extract_VGG19(train_files)
valid_vgg19 = extract_VGG19(valid_files)
test_vgg19 = extract_VGG19(test_files)
print("VGG19 shape", train_vgg19.shape[1:])

train_resnet50 = extract_Resnet50(train_files)
valid_resnet50 = extract_Resnet50(valid_files)
test_resnet50 = extract_Resnet50(test_files)
print("Resnet50 shape", train_resnet50.shape[1:])

def input_branch(input_shape=None):
    
    size = int(input_shape[2] / 4)
    
    branch_input = Input(shape=input_shape)
    branch = GlobalAveragePooling2D()(branch_input)
    branch = Dense(size, use_bias=False, kernel_initializer='uniform')(branch)
    branch = BatchNormalization()(branch)
    branch = Activation("relu")(branch)
    return branch, branch_input

vgg19_branch, vgg19_input = input_branch(input_shape=(7, 7, 512))
resnet50_branch, resnet50_input = input_branch(input_shape=(7, 7, 2048))
concatenate_branches = Concatenate()([vgg19_branch, resnet50_branch])

net = Dropout(0.3)(concatenate_branches)
net = Dense(640, use_bias=False, kernel_initializer='uniform')(net)
net = BatchNormalization()(net)
net = Activation("relu")(net)
net = Dropout(0.3)(net)
net = Dense(12, kernel_initializer='uniform', activation="softmax")(net)

model = Model(inputs=[vgg19_input, resnet50_input], outputs=[net])
model.summary()



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1200/1200 [00:16<00:00, 74.13it/s]


VGG19 shape (7, 7, 512)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7425/7425 [01:34<00:00, 78.51it/s]
d:\mlproject\vessel_classifier\.venv\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1200/1200 [00:16<00:00, 54.23it/s]


Resnet50 shape (7, 7, 2048)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 7, 7, 2048)   0                                            
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 512)          0           input_7[0][0]                    
__________________________________________________________________________________________________
global_average_pooling2d_2 (Glo (None, 2048)         0           input_8[0][0]                    
_________________________________________________________________________________

In [3]:
model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='saved_models/bestmodel_new.hdf5', 
                               verbose=1, save_best_only=True)
model.fit([train_vgg19, train_resnet50], train_targets, 
          validation_data=([valid_vgg19, valid_resnet50], valid_targets),
          epochs=20, batch_size=4, callbacks=[checkpointer], verbose=2)

model.load_weights('saved_models/bestmodel_new.hdf5')

predictions = model.predict([valid_vgg19, valid_resnet50])
breed_predictions = [np.argmax(prediction) for prediction in predictions]
breed_true_labels = [np.argmax(true_label) for true_label in test_targets]
print('Test accuracy: %.4f%%' % (accuracy_score(breed_true_labels, breed_predictions) * 100))


Train on 7425 samples, validate on 1200 samples
Epoch 1/20
 - 42s - loss: 1.0584 - acc: 0.6778 - val_loss: 0.3550 - val_acc: 0.8850

Epoch 00001: val_loss improved from inf to 0.35503, saving model to saved_models/bestmodel_new.hdf5
Epoch 2/20
 - 21s - loss: 0.8290 - acc: 0.7537 - val_loss: 0.2456 - val_acc: 0.9242

Epoch 00002: val_loss improved from 0.35503 to 0.24561, saving model to saved_models/bestmodel_new.hdf5
Epoch 3/20
 - 19s - loss: 0.7250 - acc: 0.7939 - val_loss: 0.2181 - val_acc: 0.9350

Epoch 00003: val_loss improved from 0.24561 to 0.21808, saving model to saved_models/bestmodel_new.hdf5
Epoch 4/20
 - 18s - loss: 0.6699 - acc: 0.8137 - val_loss: 0.1926 - val_acc: 0.9408

Epoch 00004: val_loss improved from 0.21808 to 0.19257, saving model to saved_models/bestmodel_new.hdf5
Epoch 5/20
 - 18s - loss: 0.6484 - acc: 0.8256 - val_loss: 0.1860 - val_acc: 0.9442

Epoch 00005: val_loss improved from 0.19257 to 0.18600, saving model to saved_models/bestmodel_new.hdf5
Epoch 6/20


In [4]:
img = path_to_tensor('data/images/doge.jpg')
preprocessed_imgvgg = preprocess_input_vgg19(img)
preprocessed_imgresnet = preprocess_input_resnet50(img)


img_vgg19 = VGG19(weights='imagenet', include_top=False).predict(preprocessed_imgvgg, batch_size=32)
img_resnet = ResNet50(weights='imagenet', include_top=False).predict(preprocessed_imgresnet, batch_size=32)

classification = model.predict([img_vgg19, img_resnet])
ship_names = [item[18:-1] for item in sorted(glob("data/images/train/*/"))]
print(ship_names[np.argmax(classification)])


fishing


In [5]:
model.save('vgg16_resnet50_ship_model_combined.h5')

In [ ]:
model = load_model('vgg16_resnet50_ship_model_combined.h5')
